In [ ]:
!pip install underthesea
!pip install textblob
!pip install wordcloud
!pip install transformers safetensors

In [25]:
df = pd.read_csv("/kaggle/input/comments-vn/Danh_gia_processing_final_training.csv")
df.head()

,id,ma_khach_hang,noi_dung_binh_luan,ngay_binh_luan,gio_binh_luan,so_sao,ma_san_pham,label_star,label_sentiment
0,1,443,sử dụng dễ dàng rất thoải mái thư giãn tột độ,29/04/2023,17:06,5,308500015,3,POS
1,2,1030,sử dụng dễ dãng rất thoải mái thư giãn tột độ,30/04/2023,15:04,5,308500015,3,POS
2,3,689,mình rất thích hasaki va sản phẩm tẩy trang này,30/04/2023,18:34,5,422216594,3,POS
3,4,2519,sản phẩm có khả năng làm sạch tốt lớp trang đi...,17/07/2022,13:48,5,204100075,3,POS
4,5,402,sữa rửa mặt tốt sạch mụn mịn da đáng mua nha,15/04/2023,23:04,5,422208977,3,POS


In [26]:
data = df[['noi_dung_binh_luan', 'label_sentiment']]
data.head()

,noi_dung_binh_luan,label_sentiment
0,sử dụng dễ dàng rất thoải mái thư giãn tột độ,POS
1,sử dụng dễ dãng rất thoải mái thư giãn tột độ,POS
2,mình rất thích hasaki va sản phẩm tẩy trang này,POS
3,sản phẩm có khả năng làm sạch tốt lớp trang đi...,POS
4,sữa rửa mặt tốt sạch mụn mịn da đáng mua nha,POS


In [27]:
data.isna().sum()

noi_dung_binh_luan    38
label_sentiment        2
dtype: int64

In [28]:
data = data.dropna()

In [29]:
data.isna().sum()

noi_dung_binh_luan    0
label_sentiment       0
dtype: int64

In [30]:
label_mapping = {'NEG': 0, 'POS': 1, 'NEU': 2}
data['label_sentiment'] = data['label_sentiment'].map(label_mapping)

In [41]:
import torch
print("GPU Available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))


GPU Available: True
Number of GPUs: 2
GPU Name: Tesla T4


In [42]:
import os
import torch
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    GenerationConfig,
)

# Vô hiệu hóa cảnh báo parallelism của tokenizer
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# **1. Chuẩn bị dữ liệu**
# label_mapping = {'NEG': 0, 'POS': 1, 'NEU': 2}
# data['label_sentiment'] = data['label_sentiment'].map(label_mapping)

# Chia dữ liệu thành train và test
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data["noi_dung_binh_luan"], data["label_sentiment"], test_size=0.2, random_state=42
)

# **2. Load Tokenizer**
model_path = "5CD-AI/Vietnamese-Sentiment-visobert"
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Tokenize dữ liệu
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=256)

# **3. Tạo Dataset Class**
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

# Tạo dataset
train_dataset = SentimentDataset(train_encodings, train_labels.tolist())
test_dataset = SentimentDataset(test_encodings, test_labels.tolist())

# **4. Load mô hình**
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=3)

if not hasattr(model, "generation_config") or model.generation_config is None:
    model.generation_config = GenerationConfig.from_model_config(model.config)


# Xóa `max_length` khỏi `model.config` nếu nó tồn tại
if hasattr(model.config, "max_length"):
    del model.config.max_length

# Đảm bảo `max_length` được gán trong `generation_config`
model.generation_config.max_length = 256
print("GenerationConfig đã được cập nhật với max_length =", model.generation_config.max_length)

# **5. Thiết lập TrainingArguments**
training_args = TrainingArguments(
    output_dir="/kaggle/working/results",      # Thư mục lưu kết quả
    eval_strategy="epoch",              # Đánh giá sau mỗi epoch
    save_strategy="epoch",                    # Lưu mô hình sau mỗi epoch
    learning_rate=2e-5,                       # Learning rate
    per_device_train_batch_size=16,           # Batch size trên mỗi GPU
    per_device_eval_batch_size=16,            # Batch size validation
    num_train_epochs=3,                       # Số epoch
    weight_decay=0.01,                        # Weight decay
    logging_dir="/kaggle/working/logs",       # Thư mục log
    load_best_model_at_end=True,              # Load model tốt nhất sau khi train
    fp16=True,                                # Mixed Precision
    report_to="none",                         # Tắt báo cáo tới W&B
    dataloader_num_workers=4,                 # Số luồng xử lý dữ liệu
    save_total_limit=2,                       # Giới hạn số lượng checkpoints
    disable_tqdm=False,                       # Hiển thị tiến trình
)

# **6. Tạo Trainer**
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

# **Khắc phục vấn đề multiprocessing**
if __name__ == "__main__":
    # **7. Huấn luyện**
    trainer.train()

    # **8. Lưu mô hình**
    trainer.save_model("/kaggle/working/fine_tuned_visobert")
    tokenizer.save_pretrained("/kaggle/working/fine_tuned_visobert")

GenerationConfig đã được cập nhật với max_length = 256


/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  wa

Epoch,Training Loss,Validation Loss
1,0.132000,0.141478
2,0.079800,0.131649
3,0.028900,0.125283


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(

In [43]:
!zip -r /kaggle/working/results.zip /kaggle/working/results


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  adding: kaggle/working/results/ (stored 0%)
  adding: kaggle/working/results/checkpoint-1032/ (stored 0%)
  adding: kaggle/working/results/checkpoint-1032/sentencepiece.bpe.model (deflated 50%)
  adding: kaggle/working/results/checkpoint-1032/tokenizer.json (deflated 78%)
  adding: kaggle/working/results/checkpoint-1032/config.json (deflated 50%)
  adding: kaggle/working/results/checkpoint-1032/tokenizer_config.json (deflated 76%)
  adding: kaggle/working/results/checkpoint-1032/scheduler.pt (deflated 55%)
  adding: kaggle/working/results/checkpoint-1032/training_args.bin (deflated 51%)
  adding: kaggle/working/results/checkpoint-1032/optimizer.pt (deflated 17%)
  adding: kaggle/working/results/checkpoint-1032/model.safetensors (deflated 7%)
  adding: kaggle/working/results/checkpoint-1032/trainer_state.json (deflated 61%)
  adding: kaggle/working/results/checkpoint-1032/special_tokens_map.json (deflated 85%)
  adding: kaggle/working/results/checkpoint-1032/rng_state.pth (deflated 25

In [44]:
!zip -r /kaggle/working/fine_tuned_visobert.zip /kaggle/working/fine_tuned_visobert

  adding: kaggle/working/fine_tuned_visobert/ (stored 0%)
  adding: kaggle/working/fine_tuned_visobert/sentencepiece.bpe.model (deflated 50%)
  adding: kaggle/working/fine_tuned_visobert/tokenizer.json (deflated 78%)
  adding: kaggle/working/fine_tuned_visobert/config.json (deflated 50%)
  adding: kaggle/working/fine_tuned_visobert/tokenizer_config.json (deflated 76%)
  adding: kaggle/working/fine_tuned_visobert/training_args.bin (deflated 51%)
  adding: kaggle/working/fine_tuned_visobert/model.safetensors (deflated 7%)
  adding: kaggle/working/fine_tuned_visobert/special_tokens_map.json (deflated 85%)
